In [2]:
import pandas as pd
import numpy as np
import pyomo
import pandas as pd
import pyomo.opt
import pyomo.environ as pe

In [39]:
MAX_EVENTS_PER_STUDENT = 4
MIN_EVENTS_PER_STUDENT = 3
MAX_STUDENTS_PER_EVENT = 3
MIN_STUDENTS_PER_EVENT = 2
PREFERENCES_FILE = "./preferences.csv"
ROOMS_FILE = "./rooms.csv"


In [40]:
def read_rooms(rooms_file: str) -> np.ndarray:
    return pd.read_csv(rooms_file, index_col=0).astype(int).to_numpy().flatten()

In [41]:
rooms = read_rooms(ROOMS_FILE)
NUM_ROOMS = rooms.shape[0]

In [42]:
def read_preferences(preferences_file: str) -> np.ndarray:
    return pd.read_csv(preferences_file, index_col=0).astype(int).to_numpy()

In [43]:
preferences = read_preferences(PREFERENCES_FILE)
NUM_RESIDENTS = preferences.shape[0]
NUM_PREFERENCE_CATEGORIES = preferences.shape[1]
preferences, NUM_RESIDENTS, NUM_PREFERENCE_CATEGORIES

(array([[2, 1, 1, 3, 1],
        [2, 1, 4, 3, 1],
        [1, 1, 4, 4, 2],
        [4, 1, 3, 4, 4],
        [3, 4, 1, 4, 4],
        [3, 3, 2, 2, 3],
        [3, 2, 4, 3, 2],
        [3, 3, 2, 1, 4],
        [2, 3, 3, 1, 4],
        [1, 4, 2, 2, 4],
        [2, 4, 4, 1, 1],
        [4, 1, 4, 2, 3],
        [3, 4, 2, 3, 1],
        [3, 2, 4, 4, 2],
        [3, 1, 3, 1, 3]]),
 15,
 5)

In [44]:
# returns array where arr[i][j] represents the compatibility between two people, calculated as the sum of the absolute difference between the two vectors (might change to square root of the sum of the squares or something later!) or maybe the normalized dot product

def calculate_compatibility_array(preferences, num_residents=-1, num_preference_categories=-1):
    def calculate_compatibility(a_prefs, b_prefs):
        return np.sum(np.abs(a_prefs - b_prefs))

    if num_residents == -1:
        num_residents = preferences.shape[0]
    if num_preference_categories == -1:
        num_preference_categories = preferences.shape[1]

    compatibilities = []

    for a, a_prefs in enumerate(preferences):
        a_compatibilities = []
        for b, b_prefs in enumerate(preferences):
            if a == b:
                a_compatibilities.append(float("inf"))
            else:
                a_compatibilities.append(calculate_compatibility(a_prefs, b_prefs))
        compatibilities.append(a_compatibilities)
    
    return np.array(compatibilities)

compatibility_array = calculate_compatibility_array(preferences)
    
    
        

In [45]:
problem = pulp.LpProblem('lpsolver', pulp.LpMinimize)

In [46]:
compatibility_array

array([[inf,  3.,  6.,  8.,  8.,  7.,  6.,  9.,  9.,  9.,  8.,  8.,  5.,
         7.,  7.],
       [ 3., inf,  3.,  7., 11.,  8.,  3., 10.,  8., 10.,  5.,  5.,  6.,
         4.,  6.],
       [ 6.,  3., inf,  6., 10.,  9.,  4., 11.,  9.,  9.,  8.,  6.,  9.,
         3.,  7.],
       [ 8.,  7.,  6., inf,  6.,  7.,  6.,  7.,  7.,  9., 12.,  4.,  9.,
         5.,  5.],
       [ 8., 11., 10.,  6., inf,  5.,  8.,  5.,  7.,  5., 10., 10.,  5.,
         7.,  9.],
       [ 7.,  8.,  9.,  7.,  5., inf,  5.,  2.,  4.,  4.,  7.,  5.,  4.,
         6.,  4.],
       [ 6.,  3.,  4.,  6.,  8.,  5., inf,  7.,  7.,  9.,  6.,  4.,  5.,
         1.,  5.],
       [ 9., 10., 11.,  7.,  5.,  2.,  7., inf,  2.,  4.,  7.,  7.,  6.,
         8.,  4.],
       [ 9.,  8.,  9.,  7.,  7.,  4.,  7.,  2., inf,  4.,  5.,  7.,  8.,
         8.,  4.],
       [ 9., 10.,  9.,  9.,  5.,  4.,  9.,  4.,  4., inf,  7.,  9.,  6.,
        10.,  8.],
       [ 8.,  5.,  8., 12., 10.,  7.,  6.,  7.,  5.,  7., inf,  8.,  5.,
       

In [47]:
# def read_schedule(schedule_file: str) -> list[list[int]]:
#     row_numbers = []
#     schedule = pd.read_csv("./schedule.csv",index_col=0).fillna(0).astype(int)
#     for col in schedule.columns:
#         row_numbers.append(schedule.loc[schedule[col] == 1].index.values.tolist())
#     return row_numbers


# SCHEDULE = read_schedule("./schedule.csv")
# SCHEDULE

In [48]:
problem = pulp.LpProblem('lpsolver', pulp.LpMinimize)

In [49]:
#create one variable for every resident-room combo (NUM_RESIDENTS * NUM_ROOMS)
x = pulp.LpVariable.dicts('x', range(NUM_RESIDENTS * NUM_ROOMS), lowBound=0, upBound=1, cat='Integer')

In [51]:
p = read_preferences(PREFERENCES_FILE) # TODO: convert p into a 2D array rather than 1D (more readable)

# problem.setObjective(sum([sum([p[NUM_EVENTS * i + j]*x[NUM_EVENTS * i + j] for j in range(NUM_EVENTS)])for i in range(NUM_STUDENTS)]))

problem.setObjective(
    sum([
        sum([
            sum([
                compatibility_array[s][t]*x[s * NUM_ROOMS + j]*x[t * NUM_ROOMS + j] 
                for s in range(NUM_RESIDENTS)]) 
            for t in range(NUM_RESIDENTS)]) 
        for j in range(NUM_ROOMS)]))
problem

/var/folders/g2/py91tm2d5bj465mlzrg0r1l40000gn/T/ipykernel_6727/3710517791.py:9: RuntimeWarning: invalid value encountered in multiply
  compatibility_array[s][t]*x[s * NUM_ROOMS + j]*x[t * NUM_ROOMS + j]


TypeError: Non-constant expressions cannot be multiplied

In [157]:
# students can do between 3 and 4 events
for student in range(NUM_STUDENTS):
    problem.addConstraint(sum([x[NUM_EVENTS * student + event] for event in range(NUM_EVENTS)]) <= MAX_EVENTS_PER_STUDENT)
    problem.addConstraint(sum([x[NUM_EVENTS * student + event] for event in range(NUM_EVENTS)]) >= MIN_EVENTS_PER_STUDENT)


In [158]:
# events have 2-3 people each
for event in range(NUM_EVENTS):
    problem.addConstraint(sum([x[NUM_EVENTS * student + event] for student in range(NUM_STUDENTS)]) >= MIN_STUDENTS_PER_EVENT)
    problem.addConstraint(sum([x[NUM_EVENTS * student + event] for student in range(NUM_STUDENTS)]) <= MAX_STUDENTS_PER_EVENT)

In [159]:
# students cannot have positive assignments to events in the same slot
for slot in SCHEDULE:
    for student in range(NUM_STUDENTS):
        problem.addConstraint(sum([x[NUM_EVENTS * student + event] for event in slot]) <= 1)

In [160]:
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Kanav/opt/anaconda3/envs/scioly/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/g2/py91tm2d5bj465mlzrg0r1l40000gn/T/d3e166f9801f4cc49e778361cc4425e6-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/g2/py91tm2d5bj465mlzrg0r1l40000gn/T/d3e166f9801f4cc49e778361cc4425e6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 201 COLUMNS
At line 2693 RHS
At line 2890 BOUNDS
At line 3236 ENDATA
Problem MODEL has 196 rows, 345 columns and 1725 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 146 - 0.00 seconds
Cgl0004I processed model has 158 rows, 345 columns (345 integer (345 of which binary)) and 1035 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -146

1

In [161]:
values = [x[i].value() for i in range(NUM_EVENTS * NUM_STUDENTS)]
all([value == 0.0 or value == 1.0 for value in values])

True

In [162]:
sum(values)

53.0

In [167]:

output = pd.DataFrame(np.array(values).reshape(15, 23)).astype(int)

In [168]:
output.to_csv("./output.csv")

In [165]:
event = 13
sum([x[NUM_EVENTS * student + event].value() for student in range(NUM_STUDENTS)])

2.0

In [166]:
slot = SCHEDULE[0]
student = 9
sum([x[NUM_EVENTS * student + event].value() for event in slot])

0.0

In [ ]:
import